In [1]:
import pandas as pd
import random
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

def fixDFcolsForR(myDF):
    for i in range(myDF.shape[1]): # for each col
        if myDF.iloc[:, i].dtype == 'O': # check if col is type object
            myDF.iloc[:, i] = myDF.iloc[:, i].astype(str) # if yes set type to string
    return myDF

# Overal Data SetManipulations

How we used this first notebook that we iterated/cleaned on the csvs here, and then saved it to a github repo. From there we could independently pull/push our cleaned and merged datasets

In [2]:
meta = pd.read_csv('./the-movies-dataset-clean/movies_metadata.csv')

/Users/nickparker/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Column Math

In [3]:
### Adding profit and use of Budget Columns
meta['profit'] = meta['revenue'] - pd.to_numeric(meta['budget'],errors='coerce')
meta['useOfBudget'] = meta['revenue'] / (pd.to_numeric(meta['budget'],errors='coerce') + 000.1)

# Splitting Columns with multiple values

In [4]:
#This function takes in a column which contains a dictionary
#It splits it by the elements of the dict into n new columsn columns
#example: 
#genre[0]=[{'name':'mystery'},{'name':'thriller'}] --> genre_1[0] = 'mystery', genre_2 = 'thriller' 
def splitDict(df, colToSplit, n):   
    for i in range(n):
        newCol = []
        for rowNum in range(df.shape[0]):
            try: 
                newCol.append(ast.literal_eval(df[colToSplit][rowNum])[i]['name'])
            except:
                newCol.append('nan')
        newColName = colToSplit + '_'+ str(i+1)
        df[newColName] = newCol
    df = df.drop(colToSplit,axis = 1)
    return df

In [5]:
meta = splitDict(meta, 'production_companies', 6)
meta = splitDict(meta, 'genres', 3)
meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'homepage', 'id', 'imdb_id',
       'original_language', 'original_title', 'overview', 'popularity',
       'poster_path', 'production_countries', 'release_date', 'revenue',
       'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'profit', 'useOfBudget',
       'production_companies_1', 'production_companies_2',
       'production_companies_3', 'production_companies_4',
       'production_companies_5', 'production_companies_6', 'genres_1',
       'genres_2', 'genres_3'],
      dtype='object')

## Merging

In [6]:
#Here we add two more datasets
#ratings, contains 26 millions reviews
ratings = pd.read_csv('./the-movies-dataset-clean/ratings.csv')
#links is a helpful dataset with movie id's and imdb id's
links = pd.read_csv('./the-movies-dataset-clean/links.csv')

#First we aggregate ratings by movie
ratings = ratings.groupby('movieId').agg({'rating':['mean','count']}).reset_index() #gets average rating
ratings.columns = ['movieId','rating','rateCounts']
#Then we merge the ratings onto links
linkAvgRating = pd.merge(links,ratings,on='movieId',how='left')

#Then we standardized the merge column (imdbID) and merge it together
meta['imdbId'] = pd.to_numeric(meta['imdb_id'].str[2:])
meta = pd.merge(meta,linkAvgRating,on='imdbId',how='left')

## Making Flops dataframe

In [7]:
#Since revenue and budget are such important columns to us,
#We remove columns with missing values (0 is an illogical budget)
flop = meta.loc[((meta['revenue'] > 0) & pd.to_numeric(meta['budget'],errors='coerce' != 0))]
flop['flop'] = 0
flop.loc[(flop['profit'] < 0),'flop'] = 1

/Users/nickparker/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/nickparker/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


## Making the df longer based on genre

In [8]:
#Here we do the equivelent of melt by making the df long to wide  for genre
long_df_genre = pd.wide_to_long(flop.reset_index(), 
                          i=['index'], 
                          j='genreNo',
                          stubnames='genres',
                          sep="_")
long_df_genre.reset_index(inplace=True)

## Making the df longer based on prod company

In [9]:
#We do the same for production company
long_df_prod = pd.wide_to_long(flop.reset_index(), 
                          i=['index'], 
                          j='companyNo',
                          stubnames='production_companies',
                          sep="_")
long_df_prod.shape

(32286, 34)

In [10]:
long_df_prod.columns

Index(['movieId', 'original_language', 'production_countries', 'useOfBudget',
       'genres_3', 'status', 'flop', 'budget', 'tmdbId', 'release_date',
       'tagline', 'vote_count', 'vote_average', 'id', 'spoken_languages',
       'adult', 'title', 'genres_1', 'overview', 'profit', 'runtime', 'video',
       'popularity', 'poster_path', 'genres_2', 'rateCounts',
       'belongs_to_collection', 'imdbId', 'revenue', 'imdb_id',
       'original_title', 'rating', 'homepage', 'production_companies'],
      dtype='object')

In [11]:
#checking it split correctly
long_df_prod.reset_index(inplace=True)

display = long_df_prod[['title','id','companyNo','production_companies']]

display.sort_values('id').head(6)

,title,id,companyNo,production_companies
17079,"Lock, Stock and Two Smoking Barrels",100,4,SKA Films
22460,"Lock, Stock and Two Smoking Barrels",100,5,The Steve Tisch Company
11698,"Lock, Stock and Two Smoking Barrels",100,3,PolyGram Filmed Entertainment
27841,"Lock, Stock and Two Smoking Barrels",100,6,nan
6317,"Lock, Stock and Two Smoking Barrels",100,2,Summit Entertainment
936,"Lock, Stock and Two Smoking Barrels",100,1,Handmade Films Ltd.


# Export

#### When we are done with the data manipulation, we saved it as csv in our Github and were able to work from there

In [12]:
#export_csv = meta.to_csv (r'movie_dataset_edited/meta.csv', header=True)
#Don't forget to add '.csv' at the end of the path

In [13]:
#export_csv = flop.to_csv (r'movie_dataset_edited/flopDataSet.csv', header=True)


In [14]:
#export_csv = long_df_genre.to_csv (r'movie_dataset_edited/flopTripleByGenre.csv', header=True)
